In [ ]:
%tensorflow_version 1.x
!pip install keras==2.1.0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd
import os
!git clone --quiet https://github.com/rvignav/Mask_RCNN.git
if not os.path.isdir('../logs'):
  os.mkdir('../logs')
!cp -r '/content/drive/My Drive/lesion' '../logs/lesion'

In [ ]:
%cd ~/Mask_RCNN

!pip install -q PyDrive
!pip install -r requirements.txt
!python setup.py install

In [ ]:
%cd ~/Mask_RCNN

!cp ~/Mask_RCNN/samples/balloon/balloon.py ./lesion.py

!sed -i -- 's/balloon/lesion/g' lesion.py
!sed -i -- 's/Balloon/Lesion/g' lesion.py
!sed -i -- 's/epochs=30/epochs=100/g' lesion.py

In [ ]:
%cd ~/Mask_RCNN

# !python lesion.py train --dataset='/content/drive/My Drive/images2/' --weights=last

In [ ]:
import cv2
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import skimage
import glob
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log
# from google.colab import files

import lesion

# Root directory of the project
ROOT_DIR = os.getcwd()

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library

custom_WEIGHTS_PATH = sorted(glob.glob("/logs/*/mask_rcnn_*.h5"))[-1]
# files.download(custom_WEIGHTS_PATH)

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

config = lesion.LesionConfig()
custom_DIR = os.path.join(ROOT_DIR, '/content/drive/My Drive/images2/')

class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()

# Device to load the neural network on.
# Useful if you're training a model on the same 
# machine, in which case use CPU and leave the
# GPU for training.
DEVICE = "/gpu:0"  # /cpu:0 or /gpu:0

# Inspect the model in training or inference modes
# values: 'inference' or 'training'
# TODO: code for 'training' test mode not ready yet
TEST_MODE = "inference"

def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax
  
# Load validation dataset
dataset = lesion.LesionDataset()
dataset.load_lesion(custom_DIR, "val")

# Must call before using the dataset
dataset.prepare()

print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

# Create model in inference mode
with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR,
                              config=config)

# load the last model you trained
# weights_path = model.find_last()[1]

# Load weights
print("Loading weights ", custom_WEIGHTS_PATH)
model.load_weights(custom_WEIGHTS_PATH, by_name=True)

from importlib import reload # was constantly changin the visualization, so I decided to reload it instead of notebook
reload(visualize)

In [ ]:
from PIL import Image

def flatten(seq):
  for el in seq:
    if isinstance(el, list):
      yield from flatten(el)
    else:
      yield el

dices = []

for i in range(2):
  AMOUNT = 0 #212-45
  image_id = dataset.image_ids[i]
  all_points = []
  f = open('/content/drive/My Drive/images2/val' + str(i) + '.txt', 'r')
  l = f.read().replace("[", "").replace("]", "").split("), ")

  minArr = 100000000
  for item in l:
    t = item[1:].split(", ")
    if (int(t[0]) < minArr):
      minArr = int(t[0])
  
  minAll = 10000000
  for item in l:
    t = item[1:].split(", ")
    if ')' in t[1]:
      t[1] = t[1][0:len(t[1])-1]
    new_tup = (int(t[1]), -1 * int(t[0]))
    if (-1 * int(t[0]) < minAll):
      minAll = -1 * int(t[0])
    if new_tup not in all_points:
      all_points.append(new_tup)

  for i in range(len(all_points)):
    all_points[i] = (all_points[i][0], all_points[i][1] + (minArr - minAll))

  image, image_meta, gt_class_id, gt_bbox, gt_mask =\
      modellib.load_image_gt(dataset, config, image_id, use_mini_mask=False)
  info = dataset.image_info[image_id]
  results = model.detect([image], verbose=1)
  r = results[0]

  masked = r['masks']

  fig=plt.figure()
  ax2=fig.add_axes([0,0,1,1])

  xArr = []
  yArr = []
  for idx in range(masked.shape[2]):
    array = masked[:,:,idx]
    xs = []
    ys = []
    for i in range(len(array)):
        for j in range(len(array[0])):
            if array[i][j] == True:
                xs.append(j)
                ys.append(i)
    xArr.append(xs)
    yArr.append(ys)

  m = max(flatten(yArr)) + min(flatten(yArr))
  for i in range(len(yArr)):
    ys = yArr[i]
    for j in range(len(ys)):
      ys[j] = m - ys[j]
    yArr[i] = ys
  
  all_masks = []
  for i in range(len(xArr)):
    for j in range(len(xArr[i])):
      tup = (xArr[i][j], yArr[i][j])
      if tup not in all_masks: 
        all_masks.append(tup)
  
  xp = []
  yp = []

  for i in range(len(all_points)):
    tup = all_points[i]
    tup = (tup[0], tup[1] + AMOUNT)
    all_points[i] = tup

  for tup in all_points:
    xp.append(tup[0])
    yp.append(tup[1])
  
  ax2.scatter(xp, yp, color=str('g'))

  for i in range(len(xArr)):
    xs = xArr[i]
    ys = yArr[i]
    ax2.scatter(xs, ys, color=str('b'))

  plt.show()

  over = 0
  for tup in all_masks:
    if tup in all_points:
      over += 1
  tot = len(all_points) + len(all_masks) - over
  dices.append(float(over)/tot)
  print("Individual DICE Score: " + str(float(over)/tot))

dice_coeff = sum(dices) / len(dices)
print("Overall DICE Coefficient: " + str(dice_coeff))

In [ ]:
for i in range(2):
  image_id = dataset.image_ids[i]
  image, image_meta, gt_class_id, gt_bbox, gt_mask =\
      modellib.load_image_gt(dataset, config, image_id, use_mini_mask=False)
  info = dataset.image_info[image_id]
  results = model.detect([image], verbose=1)
  ax = get_ax(1)
  r = results[0]

  visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                              dataset.class_names, r['scores'], ax=ax,
                              title="Predictions")